In [1]:
!pip -q install rank_bm25
!pip -q install pyvi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 50.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.9 MB/s eta 0:00:00


In [2]:
import os
import json
import pandas as pd
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk
from pyvi.ViTokenizer import tokenize

In [3]:
# Đường dẫn tới thư mục chứa các file JSON
qa_folder_path = '/kaggle/input/qa-dataset/all_dataset_qa_retrieve'

# Danh sách để chứa dữ liệu
qa_data = []

# Lặp qua tất cả các file trong thư mục và sử dụng tqdm để theo dõi tiến trình
for file_name in tqdm(os.listdir(qa_folder_path), desc="Reading files"):
    if file_name.endswith('.json'):  # Kiểm tra nếu file là file JSON
        file_path = os.path.join(qa_folder_path, file_name)
        
        # Mở và đọc dữ liệu từ file JSON
        with open(file_path, 'r', encoding='utf-8') as f:
            qa_file_data = json.load(f)
            
            # Lặp qua các câu hỏi và câu trả lời trong dữ liệu
            for qa in qa_file_data['qa']:
                qa_data.append({
                    'question': qa['question'],
                    'answer': qa['answer'],
                    'id': qa['id']
                })

Reading files: 100%|██████████| 499/499 [00:01<00:00, 309.97it/s]


In [4]:
# Tạo DataFrame từ dữ liệu đã thu thập
qa_df = pd.DataFrame(qa_data)

In [5]:
qa_df['tokenized_question'] = qa_df['question'].apply(lambda x: tokenize(x.lower()).split())

In [6]:
# Tạo DataFrame từ dữ liệu đã thu thập
chunked_df = pd.read_json('/kaggle/input/df-full-content-and-content/df/chunked_df.json')

In [9]:
def preprocess_text(text):
    # Tokenize văn bản
    return tokenize(text.lower()).split()  # Chuyển về chữ thường và tách từ

# Tiền xử lý tất cả các văn bản trong 'content'
tokenized_corpus = [preprocess_text(content) for content in chunked_df['content']]

In [10]:
# Khởi tạo BM25 với corpus đã được tiền xử lý
bm25 = BM25Okapi(tokenized_corpus)

# Hàm để thực hiện truy vấn
def query_bm25(query, bm25, top_n=5):
    # Tiền xử lý truy vấn
    tokenized_query = preprocess_text(query)
    
    # Tính điểm BM25 cho truy vấn
    scores = bm25.get_scores(tokenized_query)
    
    # Sắp xếp các tài liệu theo điểm và lấy top N kết quả
    top_doc_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_n]
    
    # Trả về các tài liệu tương ứng với top N
    top_docs = df.iloc[top_doc_indices]
    return top_docs

In [16]:
import pickle

# Lưu mô hình BM25
with open('bm25_model_pyvi.pkl', 'wb') as f:
    pickle.dump(bm25, f)

# Lưu DataFrame chứa dữ liệu
chunked_df.to_pickle('bm25_pyvi_data.pkl')


In [11]:
from tqdm import tqdm

# Hàm đánh giá BM25
def evaluate_bm25_on_questions(df, qa_df, bm25, top_k_values=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100]):
    results = {f"Top-{k}": 0 for k in top_k_values}  # Khởi tạo dictionary với các mức đánh giá Top-k
    total_queries = len(qa_df)

    for _, row in tqdm(qa_df.iterrows(), total=total_queries):
        query = row['tokenized_question']
        true_id = row['id']

        # Truy vấn BM25 trên `content`
        scores = bm25.get_scores(query)
        top_indices = scores.argsort()[::-1][:100]  # Lấy chỉ số của Top-20 kết quả

        # Lấy danh sách các id trả về
        top_ids = df.iloc[top_indices]['id'].tolist()

        # Đánh giá cho từng Top-k (Top-1, Top-3, Top-5, ...)
        for k in top_k_values:
            if true_id in top_ids[:k]:
                results[f"Top-{k}"] += 1

    # Tính độ chính xác cho mỗi Top-k
    for k in top_k_values:
        results[f"Top-{k}"] /= total_queries

    return results

# Kết quả

In [13]:
evaluation_results = evaluate_bm25_on_questions(chunked_df, qa_df, bm25)
print("BM25 Evaluation Results:", evaluation_results)

100%|██████████| 4121/4121 [1:54:07<00:00,  1.66s/it]  

BM25 Evaluation Results: {'Top-1': 0.3802475127396263, 'Top-2': 0.49284154331472946, 'Top-3': 0.5578743023537976, 'Top-4': 0.5891773841300655, 'Top-5': 0.6182965299684543, 'Top-6': 0.6428051443824314, 'Top-7': 0.6597913127881582, 'Top-8': 0.6765348216452317, 'Top-9': 0.6920650327590391, 'Top-10': 0.704198010191701, 'Top-11': 0.7180296044649357, 'Top-12': 0.7267653482164523, 'Top-13': 0.7362290706139286, 'Top-14': 0.7476340694006309, 'Top-15': 0.7532152390196554, 'Top-16': 0.7583110895413735, 'Top-17': 0.7641349187090511, 'Top-18': 0.7721426838146082, 'Top-19': 0.7757825770444067, 'Top-20': 0.7835476826013104, 'Top-21': 0.7879155544770686, 'Top-22': 0.7930114049987868, 'Top-23': 0.7966512982285853, 'Top-24': 0.8027177869449162, 'Top-25': 0.8051443824314487, 'Top-26': 0.8078136374666343, 'Top-27': 0.8112108711477797, 'Top-28': 0.8131521475370056, 'Top-29': 0.8170347003154574, 'Top-30': 0.8199466148992963, 'Top-31': 0.8223732103858287, 'Top-32': 0.8245571463237078, 'Top-33': 0.82649842271

In [14]:
results_df = pd.DataFrame(list(evaluation_results.items()), columns=["Top-K", "Accuracy"])

# Lưu DataFrame vào file CSV
output_csv_path = "bm25_pyvi_evaluation_results.csv"
results_df.to_csv(output_csv_path, index=False, encoding='utf-8')

print(f"Evaluation results saved to {output_csv_path}")

Evaluation results saved to bm25_pyvi_evaluation_results.csv


In [17]:
with open('/kaggle/working/bm25_model_pyvi.pkl', 'rb') as f:
    bm25_new = pickle.load(f)